In [ ]:
import os
import io
import cv2
import tensorflow as tf
import numpy as np
from PIL import Image
from tqdm import tqdm
import logging

logging.basicConfig(level=logging.INFO)

def create_tf_example(image_path, mask_path):
    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_image_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_image_io)
    width, height = image.size

    mask = np.array(Image.open(mask_path).convert('L'))
    mask_binary = np.where(mask > 128, 1, 0).astype(np.uint8) * 255

    mask_pil = Image.fromarray(mask_binary)
    bytes_io = io.BytesIO()
    mask_pil.save(bytes_io, format='PNG')
    encoded_mask_png = bytes_io.getvalue()

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_path.encode('utf8')])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_path.encode('utf8')])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_jpg])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=['jpeg'.encode('utf8')])),
        'image/object/mask': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_mask_png])),
    }))
    return tf_example

def create_tf_record(output_filename, image_dir, mask_dir):
    writer = tf.io.TFRecordWriter(output_filename)
    for image_filename in tqdm(os.listdir(image_dir), desc="Creating TFRecord"):
        if not image_filename.endswith('.png'):
            continue
        image_path = os.path.join(image_dir, image_filename)
        mask_filename = 'mask_' + image_filename
        mask_path = os.path.join(mask_dir, mask_filename)

        if not os.path.exists(mask_path):
            logging.warning(f"Mask file {mask_path} not found for image {image_path}, skipping.")
            continue

        try:
            tf_example = create_tf_example(image_path, mask_path)
            writer.write(tf_example.SerializeToString())
        except Exception as e:
            logging.error(f"Error processing image {image_path}: {e}", exc_info=True)
    writer.close()
    logging.info(f"Successfully created the TFRecord file: {output_filename}")

if __name__ == '__main__':
    output_filename = '/home/rk42218/DATA_SET_1024/data_split/train/old_val.record'  #
    image_dir = '/home/rk42218/DATA_SET_1024/data_split/val/images/'  # 
    mask_dir = '/home/rk42218/DATA_SET_1024/data_split/val/masks/'  # 
    create_tf_record(output_filename, image_dir, mask_dir)


In [ ]:
import os
import io
import cv2
import tensorflow as tf
import numpy as np
from PIL import Image
from tqdm import tqdm
import logging

logging.basicConfig(level=logging.INFO)

def extract_bounding_boxes(mask):
    """Extracts bounding boxes from a binary mask."""
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        xmin = x
        xmax = x + w
        ymin = y
        ymax = y + h
        boxes.append((xmin, ymin, xmax, ymax))
    return boxes

def create_tf_example(image_path, mask_path):
    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_jpg = fid.read()
    image = Image.open(io.BytesIO(encoded_jpg))
    width, height = image.size

    mask = np.array(Image.open(mask_path).convert('L'))
    mask_binary = np.where(mask > 128, 255, 0).astype(np.uint8)
    boxes = extract_bounding_boxes(mask_binary)

    source_id = os.path.basename(image_path).encode('utf8')

    feature = {
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[source_id])),  # Assuming you want the filename as the ID
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[source_id])),  # Storing filename as source ID
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_jpg])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'jpeg'])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=[b[0] / width for b in boxes])),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=[b[2] / width for b in boxes])),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=[b[1] / height for b in boxes])),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=[b[3] / height for b in boxes])),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

def create_tf_record(output_filename, image_dir, mask_dir):
    """Creates a TFRecord file from given image and mask directories."""
    writer = tf.io.TFRecordWriter(output_filename)
    for image_filename in tqdm(os.listdir(image_dir), desc="Creating TFRecord"):
        if not image_filename.endswith('.png'):
            continue
        image_path = os.path.join(image_dir, image_filename)
        mask_filename = 'mask_' + image_filename
        mask_path = os.path.join(mask_dir, mask_filename)

        if not os.path.exists(mask_path):
            logging.warning(f"Mask file {mask_path} not found for image {image_path}, skipping.")
            continue

        try:
            tf_example = create_tf_example(image_path, mask_path)
            writer.write(tf_example.SerializeToString())
        except Exception as e:
            logging.error(f"Error processing image {image_path}: {e}", exc_info=True)

    writer.close()
    logging.info(f"Successfully created the TFRecord file: {output_filename}")

if __name__ == '__main__':
    output_filename = '/home/rk42218/DATA_SET_1024/data_split/test/test.record'  
    image_dir = '/home/rk42218/DATA_SET_1024/data_split/test/images/'  
    mask_dir = '/home/rk42218/DATA_SET_1024/data_split/test/masks/'  
    create_tf_record(output_filename, image_dir, mask_dir)
